In [3]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import torchvision
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms, utils
import numpy as np
import pandas as pd
import mmap
import os
import json
import struct
import datetime

### Own imports

In [4]:
from Vocab_class import TermVocab
from generate_vocab import create_vocab

collection_dir = '/media/harsh/Common/IITD/COL764-IR/Project/spotify_million_playlist_dataset/data/'

In [5]:
# def create_vocab(coll_dir, TVocab :type(TermVocab()) = TermVocab()):
#     limit_file_read_to = 1
    
#     print('Creating Track Vocabulary')
#     file_count = 0
#     for filename in os.listdir(coll_dir):
#         file_count += 1
#         #print('For File:', filename)
#         with open(os.path.join(coll_dir, filename), 'r') as f:
#             mm = mmap.mmap(f.fileno(),length = 0, access = mmap.ACCESS_READ)
#             playlists = json.load(mm)['playlists']
#             # print('total playlists:', len(playlists))
#             for playlist in playlists[:4]:
#                 tracks = playlist['tracks']
#                 #print('pl:', playlist['pid'])
#                 # print('\tTotal Tracks:', len(tracks))
#                 for track in tracks[:3]:
#                     TVocab.add_track(track['track_uri'])
#                     # print('\tPOS :',track['pos'])
#                     # print('\tName: ',track['track_name'])
#                     # print('\tURI : ',track['track_uri'])
#             mm.close()
#         if file_count >= limit_file_read_to:
#             break
#     return TVocab

TVocab, idf = create_vocab(collection_dir)

Creating Track Vocabulary


In [6]:
len(TVocab.track2index)

29386

In [7]:
## Saving cells of matrix as float so 1 float value takes 4 bytes
class CoOcc:
    def __init__(self, collection_dir, TVocab:type(TermVocab()) = None,idf = None, prior = 0.0, co_occour_file = 'hpc_results/co_occ_file_5.txt'):
        #Consider top 200 as test and rest playlist as train
        self.validation_pl_upto = 200
        self.limit_file_read_to = 5
        self.coll_dir = collection_dir
        self.TVocab = TVocab
        self.idf = idf
        if TVocab is None:
            self.TVocab, self.idf = create_vocab(collection_dir)
        self.prior = prior
        self.co_occour_file = co_occour_file
        self.cell_size = 4  #size of i cell of matrix in bytes(float = 4 bytes)
    
    def f_t_b_arr(self, val):
        return bytearray(struct.pack('f', val))
    def b_arr_t_f(self, arr):
        return struct.unpack('f', arr)[0]
    
    def fill_default_values(self):
        t1 = datetime.datetime.now()
        print('writing initial values')
        n = self.TVocab.vocab_length
        c = self.prior
        cont_len = int(n*(n+1)/2)
        print('conte length      :',cont_len)
        content_float_arr = [c]*cont_len
        content_byte_arr = self.f_t_b_arr(content_float_arr[0])
        # for i in range(1, cont_len):
        #     content_byte_arr.extend(self.f_t_b_arr(content_float_arr[i]))
        #print('Conetnt byte arrat:',content_byte_arr)
        #print('byte array len    :', len(content_byte_arr))
        #print('for file          :', self.co_occour_file)
        
        ## OLD CODE:
        # f = open(self.co_occour_file, 'wb')
        # f.write((cont_len*4)*b'\0')
        # f.close()
        
        ##MODIFIED:
        with open(self.co_occour_file, '+wb') as f:
            for i in range(cont_len):
                f.write(4*b'\0')
        
        # with open(self.co_occour_file, 'r+b') as file:
        #     mm = mmap.mmap(file.fileno(),length = 0, access = mmap.ACCESS_WRITE)
        #     mm.write(content_byte_arr)
        #     mm.close()
        print('Initialization done for %s'%(self.co_occour_file))
        t2 = datetime.datetime.now()
        print('time taken', t2-t1)
        return True
    
    #Not Needed Now
    def read_co_occour_file(self):
        contents_array = []
        with open(self.co_occour_file, 'r') as file:
            mm = mmap.mmap(file.fileno(), length = 0, access = mmap.ACCESS_READ)
            content = mm.read()
            print('Content', content)
            print('f vals')
            for i in range(0, len(content), 4):
                f_val = self.b_arr_t_f(content[i:i+4])
                print(f_val, end=' ')
                contents_array.append(f_val)
            mm.close()
        return contents_array
    
    def get_file_position(self,track_i, track_j):
        i_tem = self.TVocab.to_index(track_i)
        j_tem = self.TVocab.to_index(track_j)
        #print(i_tem, track_i)
        #print(j_tem, track_j)
        if i_tem == -1 or j_tem == -1:
            #print('can\'t get pos for %s, %s in file, not present in vocab'%(track_i, track_j))
            return -1
        i, j = min(i_tem, j_tem), max(i_tem, j_tem)
        pos_i = i*self.TVocab.vocab_length - i*(i-1)/2
        pos = int(pos_i + j-i) * self.cell_size
        return pos
    
    def get_sim_track_ij(self, track_i = None, track_j = None):
        val = 0
        pos = self.get_file_position(track_i, track_j)
        if pos == -1:
            return 0
        #print('pos', pos)
        with open(self.co_occour_file,'r+b') as file:
            with mmap.mmap(file.fileno(), 0) as mm:
                #print('val before :',self.b_arr_t_f(mm[pos:pos+4]))
                val = self.b_arr_t_f(mm[pos:pos + 4])
                #print('val',val)
        return val
    
    def set_track_ij(self, track_i = None, track_j = None, val = 5.1):
        pos = self.get_file_position(track_i, track_j)
        if pos == -1:
            return False
        #print('pos',pos)
        with open(self.co_occour_file,'r+b') as file:
            with mmap.mmap(file.fileno(), 0) as mm:
                f_val = self.f_t_b_arr(val)
                #print('val before:',self.b_arr_t_f(mm[pos:pos + 4]))
                mm[pos:pos + 4] = f_val
                #print('val after :',self.b_arr_t_f(mm[pos:pos + 4]))
        return True
    
    def set_pos(self, mm, pos, val):
        if pos == -1:
            return False
        f_val = self.f_t_b_arr(val)
        mm[pos: pos + 4] = f_val
        return
    
    def get_pos(self, mm, pos):
        if pos == -1:
            return 0
        val = self.b_arr_t_f(mm[pos: pos + 4])
        return val
    
    def fill_matrix(self):
        coll_dir = self.coll_dir
        file_count = 0
        limit_file_read_to = self.limit_file_read_to
        print('Filling default(0.0) values in matrix')
        self.fill_default_values()
        print('Creating matrix-------------')
        file_c = open(self.co_occour_file,'r+b')
        mm_c = mmap.mmap(file_c.fileno(), 0)
        for filename in sorted(os.listdir(coll_dir)):
            file_count += 1
            print('For File:', filename)
            with open(os.path.join(coll_dir, filename), 'r') as f:
                with mmap.mmap(f.fileno(),length = 0, access = mmap.ACCESS_READ) as mm:
                    playlists = json.load(mm)['playlists']
                    # print('total playlists:', len(playlists))
                    for playlist in playlists[self.validation_pl_upto:]:
                        tracks = playlist['tracks'][:]
                        n = len(tracks)
                        for i in range(n):
                            for j in range(i+1,n):
                                pos = self.get_file_position(tracks[i]['track_uri'], tracks[j]['track_uri'])
                                if pos == -1:
                                    print('still some unknown tracks encountered')
                                    continue
                                val = self.get_pos(mm_c, pos)
                                self.set_pos(mm_c, pos, val + 1)
                                #val = self.get_sim_track_ij(tracks[i]['track_uri'], tracks[j]['track_uri'])
                                #self.set_track_ij(tracks[i]['track_uri'], tracks[j]['track_uri'], val + 1)
                                
            if file_count >= limit_file_read_to:
                break
        mm_c.close()
        file_c.close()
        print('Matrix created-------------')
        return True
    
    def get_row_termi(self, track_i):
        sim_track_i = []
        file_c = open(self.co_occour_file,'r+b')
        mm_c = mmap.mmap(file_c.fileno(), 0)
        for i in range(self.TVocab.vocab_length):
            track_j = self.TVocab.to_track(i)
            pos = self.get_file_position(track_i, track_j)
            val = 0
            if pos != -1:
                #print('Pos, track, index',pos, track_j, i)
                val = self.get_pos(mm_c, pos)
            sim_track_i.append(val)
        mm_c.close()
        file_c.close()
        return sim_track_i

In [8]:
co = CoOcc(collection_dir, TVocab= None, idf = None, co_occour_file='hpc_results/co_occ_file_5.txt')

Creating Track Vocabulary


In [7]:
# co.fill_matrix()

In [14]:
#print(TVocab.to_track(5))
#len(co.get_row_termi(TVocab.to_track(5)))

In [15]:
#co.get_sim_track_ij(TVocab.to_track(500), TVocab.to_track(40))

In [11]:
# mat = np.zeros((6,6))
# for i in range(6):
#     for j in range(i, 6):
#         mat[i][j] = co.get_sim_track_ij(TVocab.index2track[i], TVocab.index2track[j])

# print(mat)

In [24]:
# # Checking get_file_position method
# for i in range(6):
#     for j in range(i,6):
#         print('For x,y: %d, %d'%(i,j))
#         pos = co.get_file_position(TVocab.index2track[i], TVocab.index2track[j])
#         print('\t', pos)

For x,y: 0, 0
	 0
For x,y: 0, 1
	 4
For x,y: 0, 2
	 8
For x,y: 0, 3
	 12
For x,y: 0, 4
	 16
For x,y: 0, 5
	 20
For x,y: 1, 1
	 24
For x,y: 1, 2
	 28
For x,y: 1, 3
	 32
For x,y: 1, 4
	 36
For x,y: 1, 5
	 40
For x,y: 2, 2
	 44
For x,y: 2, 3
	 48
For x,y: 2, 4
	 52
For x,y: 2, 5
	 56
For x,y: 3, 3
	 60
For x,y: 3, 4
	 64
For x,y: 3, 5
	 68
For x,y: 4, 4
	 72
For x,y: 4, 5
	 76
For x,y: 5, 5
	 80


In [139]:
# import sys
# sys.stdout.flush()

In [147]:
# f = open('co_occour2.txt', 'r')
# print(f.read())
# f.close()
    

In [30]:
# def f_t_i_arr(val):
#     return bytearray(struct.pack('f', val))
# def i_arr_t_f(arr):
#     return struct.unpack('f', arr)[0]

In [31]:
# b_arr = f_t_i_arr(5.1)
# # print(b_arr)

bytearray(b'33\xa3@')


In [32]:
# val = i_arr_t_f(b_arr)
# print(val)
# print(type(val))

5.099999904632568
<class 'float'>
